In [2]:
import os
import pandas as pd
import numpy as np
import json
import subprocess
from Bio import SeqIO
import plotnine
import dask
from dask import delayed
from dask.distributed import Client
from itertools import combinations
import difflib
import scipy
import random
import pysais
from difflib import SequenceMatcher
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
import ahocorasick
from ahocorasick import Automaton
import sys

In [3]:
EUKulele_MAD_cds_marmmetsp=pd.read_csv("/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic-eLife/snakemake-workflows/eukulele-snake/EUKulele_marmmetsp_NB_MAD_cds_better_diatom/taxonomy_estimation/MAD.fasta.transdecoder-estimated-taxonomy.out",
                                            sep="\t")

import time
import itertools
original_fasta = "/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic_paper_narragansett/intermediate-files/04-compare/13-MAD-proteins/MAD.fasta.transdecoder.pep"
behaved_fasta = "nb_fasta_file.fasta"
os.system(" ".join(["bash","/vortexfs1/omics/alexander/akrinos/2022-euk-diversity/code/remove_newlines.sh",original_fasta,">",behaved_fasta]))

counter=1
start_time=time.time()
sequence_dict=dict()
with open(behaved_fasta,"r") as f:
    counter=1
    for line1,line2 in itertools.zip_longest(*[f]*2):
        sequence_dict[line1.split(">")[1].split(" ")[0]]=line2
        #if counter > 2000000:
        #    break
        counter=counter+1
    print(time.time()-start_time)
    
all_sequences_nb=pd.DataFrame({"transcript_name":list(sequence_dict.keys())})

merged_eukulele=EUKulele_MAD_cds_marmmetsp.merge(all_sequences_nb,how="outer")
merged_eukulele[["Domain","Division","Phylum","Class","Order",
       "Family","Genus","Species"]] = merged_eukulele.full_classification.str.split(pat="; ",n=8,expand=True)

14.619285821914673


In [5]:
merged_eukulele["Revised_name"] = [str(curr).replace(".","N") for curr in merged_eukulele["transcript_name"]]

In [80]:
len(merged_eukulele.index)

7624296

In [14]:
def lenset(input_list):
    return len(set(input_list))

def polish_input_frame(input_dir,cluster_threshold,kmer_no,sample_id,sample_code)
    curr_concat_read_in=pd.read_csv(os.path.join(input_dir,sample_id+"_kmerlen_"+str(kmer_no)+\
                                    "_dist_"+str(cluster_threshold)+"_"+str(sample_code)+".progress.csv"))

    curr_concat_read_in["sample_code"] = [1 if sample_id in curr else 0 for curr in \
                                  list(curr_concat_read_in.Source_ID)]
    tester_curr=curr_concat_read_in.groupby(["cluster_blob","HomClust"])[["Division","Domain","Family","Class","sample_code"]].\
        agg([set,lenset,len,sum]).reset_index()
    tester_curr.columns = tester_curr.columns.map('|'.join).str.strip('|')
    
    merged_w_eukulele=merged_eukulele.merge(curr_concat_read_in.\
                                  rename({"Source_ID":"Revised_name"},
                                         axis="columns")[["Revised_name","HomClust",
                                                          "cluster_blob"]].\
                                            merge(tester_curr[["Revised_name"]]),
                left_on="Revised_name",right_on="Revised_name",how="outer")
    
    return curr_concat_read_in,tester_curr,merged_w_eukulele

def build_upset_df(tester_curr,cluster_threshold):
    
    to_check=tester_curr.loc[[(("Rhizosoleniaceae" in curr)|("Skeletonemataceae" in curr)|\
                              ("Thalassiosiraceae" in curr)|("Hemiaulaceae" in curr))&\
                             (np.nan in curr) for curr in tester_curr["Family|set"]]]

    all_set=set()
    #for curr in to_check["Family|set"]:
    #    all_set.update(curr)
    #all_list_set = list(all_set)
    all_list_set = list(set(["Rhizosoleniaceae","Hemiaulaceae","Skeletonemataceae",
                    "Thalassiosiraceae"] #+ list(set(tax_table.loc[tax_table.Class=="Bacillariophyta","Family"]))))#[curr.strip() for curr in list(all_set)]

    all_ochrophyte=list(set(tax_table.loc[tax_table.Division=="Ochrophyta","Family"]))
    build_upset_df = pd.DataFrame()
    counter = 0
    for curr in range(len(to_check.index)):
        curr_df = pd.DataFrame()
        for curr_tax in all_list_set:
            curr_tax_label=curr_tax
            if curr_tax != curr_tax:
                curr_tax_label = "Sample_NB"
            if curr_tax in to_check.iloc[curr].loc["Family|set"]:
                curr_df[curr_tax_label] = [True]
            else:
                curr_df[curr_tax_label] = [False]
        curr_df["number_seqs"] = to_check.iloc[curr].loc["sample_code|sum"]
        if len(to_check.iloc[curr].loc["Family|set"].difference(all_list_set)) > 1:
            curr_df["non-diatom"] = [True]
        else:
            curr_df["non-diatom"] = [False]

        if len(to_check.iloc[curr].loc["Family|set"].difference(all_ochrophyte)) > 1:
            curr_df["non-ochrophyte"] = [True]
        else:
            curr_df["non-ochrophyte"] = [False]

        if len(to_check.iloc[curr].loc["Domain|set"].difference(set(["Eukaryota",np.nan]))) > 0:
            curr_df["non-eukaryote"] = [True]
        else:
            curr_df["non-eukaryote"] = [False]
        build_upset_df = pd.concat([build_upset_df,curr_df])
        #if counter > 100:
        #    break
        counter=counter+1
    non_target_diatom=list(set(tax_table.loc[tax_table.Class=="Bacillariophyta","Family"])-
     set(["Rhizosoleniaceae","Hemiaulaceae","Skeletonemataceae","Thalassiosiraceae",
          "Stephanodiscaceae","Chaetocerotales"]))
    build_upset_df["other_diatom"]=build_upset_df[non_target_diatom].any(axis=1)

    build_upset_df = build_upset_df.loc[:,~build_upset_df.columns.duplicated()].copy().\
        groupby(["Rhizosoleniaceae","Hemiaulaceae","Skeletonemataceae",
                 "Thalassiosiraceae","Stephanodiscaceae","Chaetocerotales"]+\
                ["non-diatom","non-eukaryote","non-ochrophyte","other_diatom"])\
        ["number_seqs"].sum()
    
    build_upset_df["Dist"] = cluster_threshold
    return build_upset_df

In [15]:
%%time
curr_concat_read_in0_5,tester_curr0_5,merged_w_eukulele0_5=polish_input_frame(input_dir="/vortexfs1/omics/alexander/akrinos/2022-euk-diversity/code/tax-aliquots/",
                                                                              cluster_threshold=0.5,
                                                                              kmer_no=3,sample_id="NarBay",sample_code="diatomsall")

CPU times: user 12min 24s, sys: 37.4 s, total: 13min 1s
Wall time: 13min 3s


In [21]:
(2237773-1686204)/1686204

0.3271069218196612

In [22]:
(1166048-993785)/993785

0.17334031002681666

In [25]:
(3691418-2920446)/2920446

0.26399118490805856

In [24]:
tester_curr0_5.loc[[np.nan in curr for curr in tester_curr0_5["Division|set"]]]["sample_code|sum"].sum()

3691418

In [26]:
1166048/3691418

0.3158807807731338

In [101]:
def aggset(input_set):
    return("_".join(sorted(list(set([str(curr) for curr in input_set if curr==curr])))))

def get_compare_agg(merged_w_eukulele,sample_id):
    merged_w_eukulele=merged_w_eukulele.loc[[sample_id in curr for curr in merged_w_eukulele["Revised_name"]]]
    comparison_frame = merged_w_eukulele.loc[(merged_w_eukulele["Domain|sum"]==merged_w_eukulele["Domain|sum"])&\
                          ((merged_w_eukulele["transcript_name"]==merged_w_eukulele["transcript_name"]))]
    comparison_frame["Family|agg"] = [aggset(curr) for curr in comparison_frame["Family|set"]]
    comparison_frame["FamilyInFill"] = ["Unannotated" if (curr!=curr)|(curr is None) else curr for curr in comparison_frame["Family"]]
    compare_agg=comparison_frame.groupby(["FamilyInFill","Family|agg"])["transcript_name"].count().reset_index().\
        sort_values("transcript_name",ascending=False)
    compare_agg["in_clust"] = ["yes,one" if (curr!="")&("_" not in curr) else "yes,two" if\
                               (curr!="")&("_" in curr) else "no" for curr in compare_agg["Family|agg"]]
    
    return compare_agg

In [32]:
pd.set_option('display.max_rows', 1000)

compare_agg=comparison_frame.groupby(["Family","Family|agg"])["transcript_name"].count().reset_index().\
    sort_values("transcript_name",ascending=False)
compare_agg.loc[compare_agg.transcript_name>1].head(n=1000)

,Family,Family|agg,transcript_name
50122,Skeletonemataceae,Skeletonemataceae,260206
44249,Prorocentraceae,Prorocentraceae,109087
53033,Suessiaceae,,71006
48344,Skeletonemataceae,,67269
25345,Hemiaulaceae,,62527
4301,Chaetocerotales,,43326
57213,Thalassiosiraceae,,40511
33303,Leptocylindraceae,Leptocylindraceae,38884
50142,Skeletonemataceae,Skeletonemataceae_Thalassiosiraceae,37502
9304,Chrysochromulinaceae,,36628


In [37]:
comparison_frame["FamilyInFill"] = ["Unannotated" if (curr!=curr)|(curr is None) else curr for curr in comparison_frame["Family"]]
compare_agg=comparison_frame.groupby(["FamilyInFill","Family|agg"])["transcript_name"].count().reset_index().\
    sort_values("transcript_name",ascending=False)
compare_agg.loc[compare_agg.transcript_name>1].head(n=1000)

<ipython-input-37-edaf94389243>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,FamilyInFill,Family|agg,transcript_name
62217,Unannotated,,1326356
50122,Skeletonemataceae,Skeletonemataceae,260206
44249,Prorocentraceae,Prorocentraceae,109087
53033,Suessiaceae,,71006
48344,Skeletonemataceae,,67269
25345,Hemiaulaceae,,62527
4301,Chaetocerotales,,43326
57213,Thalassiosiraceae,,40511
33303,Leptocylindraceae,Leptocylindraceae,38884
50142,Skeletonemataceae,Skeletonemataceae_Thalassiosiraceae,37502


In [38]:
compare_agg["in_clust"] = ["yes" if curr!="" else "no" for curr in compare_agg["Family|agg"]]

compare_agg.loc[compare_agg.FamilyInFill=="Unannotated"].groupby("in_clust").transcript_name.sum().reset_index()
compare_agg["in_clust"] = ["yes,one" if (curr!="")&("_" not in curr) else "yes,two" if\
                               (curr!="")&("_" in curr) else "no" for curr in compare_agg["Family|agg"]]

compare_agg.loc[compare_agg.FamilyInFill=="Unannotated"].groupby("in_clust").transcript_name.sum().reset_index()

,in_clust,transcript_name
0,no,1326356
1,"yes,one",25365
2,"yes,two",73841


In [41]:
compare_agg.loc[(compare_agg.FamilyInFill=="Unannotated")&\
                (compare_agg.in_clust=="yes,one")].sort_values("transcript_name",ascending=False)

,FamilyInFill,Family|agg,transcript_name,in_clust
68302,Unannotated,Hemiaulaceae,5000,"yes,one"
68865,Unannotated,Skeletonemataceae,2545,"yes,one"
65299,Unannotated,Chaetocerotales,1336,"yes,one"
68917,Unannotated,Thalassiosiraceae,1283,"yes,one"
68905,Unannotated,Suessiaceae,1273,"yes,one"
68537,Unannotated,Leptocylindraceae,1024,"yes,one"
68802,Unannotated,Prorocentraceae,882,"yes,one"
67701,Unannotated,Goniodomataceae,630,"yes,one"
68883,Unannotated,Stephanodiscaceae,610,"yes,one"
66379,Unannotated,Chrysochromulinaceae,573,"yes,one"


In [42]:
compare_agg.loc[(compare_agg.FamilyInFill=="Unannotated")&\
                (compare_agg.in_clust=="yes,two")].sort_values("transcript_name",ascending=False)

,FamilyInFill,Family|agg,transcript_name,in_clust
62340,Unannotated,Amphidomataceae_Amphipleuraceae_Aulacoseiracea...,1956,"yes,two"
62228,Unannotated,Acanthoecida_X_Amphipleuraceae_Aulacoseiraceae...,812,"yes,two"
62369,Unannotated,Amphidomataceae_Blepharismidae_Calcidiscaceae_...,785,"yes,two"
62344,Unannotated,Amphidomataceae_Aulacoseiraceae_Chlorarachnida...,606,"yes,two"
64374,Unannotated,Bicoecaceae_Chlamydomonadales_X_Chlorarachnida...,535,"yes,two"
...,...,...,...,...
68602,Unannotated,Mesodiniidae_Rhodospirillaceae,1,"yes,two"
68604,Unannotated,Mesodiniidae_Synchromaceae,1,"yes,two"
68610,Unannotated,Microcoleaceae_Prochloraceae_Synechococcaceae,1,"yes,two"
68612,Unannotated,Morganellaceae_Shewanellaceae_Vibrionaceae,1,"yes,two"


In [121]:
merged_w_eukulele0_5.head()

,Unnamed: 0,transcript_name,classification_level,full_classification,classification,max_pid,ambiguous,counts,Domain,Division,...,Family|len,Family|sum,Class|set,Class|lenset,Class|len,Class|sum,sample_code|set,sample_code|lenset,sample_code|len,sample_code|sum
0,0.0,megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta.p2,class,Eukaryota; Amoebozoa; Conosa; Variosea,Variosea,124.0,0.0,NaN,Eukaryota,Amoebozoa,...,9.0,0,{nan},1.0,9.0,0,{1},1.0,9.0,9.0
1,0.0,megahit_NarBay_A_k51_1000010_flag=1_multi=6.2333_len=1401_NarBay_A_merged.fasta.p1,order,Eukaryota; Stramenopiles; Ochrophyta; Bacillariophyta; Leptocylindrales,Leptocylindrales,322.0,0.0,NaN,Eukaryota,Stramenopiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta.p1,class,Eukaryota; Hacrobia; Cryptophyta; Cryptophyceae,Cryptophyceae,109.0,0.0,NaN,Eukaryota,Hacrobia,...,7.0,0,{nan},1.0,7.0,0,{1},1.0,7.0,7.0
3,0.0,megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta.p1,genus,Eukaryota; Hacrobia; Haptophyta; Prymnesiophyceae; Prymnesiales; Chrysochromulinaceae; Chrysochr...,Chrysochromulina,493.0,0.0,NaN,Eukaryota,Hacrobia,...,8.0,0,{nan},1.0,8.0,0,{1},1.0,8.0,8.0
4,0.0,megahit_NarBay_A_k51_1000054_flag=1_multi=2.0000_len=353_NarBay_A_merged.fasta.p1,class,Eukaryota; Stramenopiles; Ochrophyta; Chrysophyceae,Chrysophyceae,94.4,0.0,NaN,Eukaryota,Stramenopiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
def aggset(input_set):
    if input_set!=input_set:
        return "NoDeepClustWDiatom"
    return("_".join(sorted(list(set([str(curr) for curr in input_set if curr==curr])))))

merged_w_eukulele0_5["Family|agg"] = [aggset(curr) for curr in merged_w_eukulele0_5["Family|set"]]
merged_w_eukulele0_5["Division|agg"] = [aggset(curr) for curr in merged_w_eukulele0_5["Division|set"]]

merged_w_eukulele0_5[["Family|agg","Family|lenset","Division","Domain","Family","Family|set",
                      "Division|set","Class|set","transcript_name"]].head()

,Family|agg,Family|lenset,Division,Domain,Family,Family|set,Division|set,Class|set,transcript_name
0,,1.0,Amoebozoa,Eukaryota,None,{nan},{nan},{nan},megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta.p2
1,NoDeepClustWDiatom,NaN,Stramenopiles,Eukaryota,None,NaN,NaN,NaN,megahit_NarBay_A_k51_1000010_flag=1_multi=6.2333_len=1401_NarBay_A_merged.fasta.p1
2,,1.0,Hacrobia,Eukaryota,None,{nan},{nan},{nan},megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta.p1
3,,1.0,Hacrobia,Eukaryota,Chrysochromulinaceae,{nan},{nan},{nan},megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta.p1
4,NoDeepClustWDiatom,NaN,Stramenopiles,Eukaryota,None,NaN,NaN,NaN,megahit_NarBay_A_k51_1000054_flag=1_multi=2.0000_len=353_NarBay_A_merged.fasta.p1


In [131]:
merged_count_eukulele = merged_w_eukulele0_5.groupby(["Family|agg","Family","Division","Division|agg"])\
    ["transcript_name"].\
    count().reset_index().sort_values("transcript_name",ascending=False)
merged_count_eukulele.loc[merged_count_eukulele["Family|agg"] != merged_count_eukulele["Family"]]

,Family|agg,Family,Division,Division|agg,transcript_name
58602,NoDeepClustWDiatom,Prorocentraceae,Alveolata,NoDeepClustWDiatom,487083
58645,NoDeepClustWDiatom,Suessiaceae,Alveolata,NoDeepClustWDiatom,92780
268,,Suessiaceae,Alveolata,,71006
249,,Skeletonemataceae,Stramenopiles,,67268
124,,Hemiaulaceae,Stramenopiles,,62527
...,...,...,...,...,...
37153,Chlorophyta_XXX_Compsopogonales_X_Stylonematales-Group-II,Vaucheriaceae,Stramenopiles,Chlorophyta_Rhodophyta,1
37151,Chlorophyta_XXX_Compsopogonales_X_Stylonematales-Group-II,Acanthoecida_X,Opisthokonta,Chlorophyta_Rhodophyta,1
37150,Chlorophyta_XXX_Compsopogonales_X_Porphyridiaceae_Porphyridiales_X_Rhodellales_X_Stylonematales-...,Chloropicaceae,Archaeplastida,Chlorophyta_Rhodophyta,1
37147,Chlorophyta_XXX_Compsopogonales_X_Erythropeltidales_X_Porphyridiaceae_Stylonematales-Group-II,Stygamoebida_X,Amoebozoa,Chlorophyta_Rhodophyta,1


In [149]:
merged_w_eukulele0_5["Family"]=["NoEUKulele" if (curr!=curr)|(curr is None) else curr for curr in merged_w_eukulele0_5["Family"]]
merged_w_eukulele0_5["Division"]=["NoEUKulele" if (curr!=curr)|(curr is None) else curr for curr in merged_w_eukulele0_5["Division"]]

merged_count_eukulele = merged_w_eukulele0_5.groupby(["Division|agg","Family|agg","Division","Family"])\
    ["transcript_name"].\
    count().reset_index().sort_values("transcript_name",ascending=False)
merged_count_eukulele.loc[merged_count_eukulele["Family|agg"] != merged_count_eukulele["Family"]]

merged_count_eukulele.loc[merged_count_eukulele["Family|agg"] != merged_count_eukulele["Family"]].head(n=100)

,Division|agg,Family|agg,Division,Family,transcript_name
75673,NoDeepClustWDiatom,NoDeepClustWDiatom,NoEUKulele,NoEUKulele,1699261
75578,NoDeepClustWDiatom,NoDeepClustWDiatom,Alveolata,NoEUKulele,658805
261,,,Stramenopiles,NoEUKulele,600996
75588,NoDeepClustWDiatom,NoDeepClustWDiatom,Alveolata,Prorocentraceae,487083
46,,,Alveolata,NoEUKulele,281883
149,,,NoEUKulele,NoEUKulele,149211
75761,NoDeepClustWDiatom,NoDeepClustWDiatom,Stramenopiles,NoEUKulele,129204
75667,NoDeepClustWDiatom,NoDeepClustWDiatom,Hacrobia,NoEUKulele,93942
75593,NoDeepClustWDiatom,NoDeepClustWDiatom,Alveolata,Suessiaceae,92780
143,,,Hacrobia,NoEUKulele,79769


In [154]:
merged_count_eukulele.loc[#(merged_count_eukulele["Family|agg"] != merged_count_eukulele["Family"])&\
                          #(merged_count_eukulele["Family"]=="NoEUKulele")&\
                          ((merged_count_eukulele["Division"]=="Stramenopiles")|\
                           (merged_count_eukulele["Division"]=="NoEUKulele"))].head(n=100)

,Division|agg,Family|agg,Division,Family,transcript_name
75673,NoDeepClustWDiatom,NoDeepClustWDiatom,NoEUKulele,NoEUKulele,1699261
261,,,Stramenopiles,NoEUKulele,600996
92184,Ochrophyta,Skeletonemataceae,Stramenopiles,Skeletonemataceae,260206
149,,,NoEUKulele,NoEUKulele,149211
75761,NoDeepClustWDiatom,NoDeepClustWDiatom,Stramenopiles,NoEUKulele,129204
272,,,Stramenopiles,Skeletonemataceae,67268
254,,,Stramenopiles,Hemiaulaceae,62527
75766,NoDeepClustWDiatom,NoDeepClustWDiatom,Stramenopiles,Pelagomonadaceae,56149
239,,,Stramenopiles,Chaetocerotales,43326
281,,,Stramenopiles,Thalassiosiraceae,40511


In [ ]:
merged_count_eukulele.loc[#(merged_count_eukulele["Family|agg"] != merged_count_eukulele["Family"])&\
                          #(merged_count_eukulele["Family"]=="NoEUKulele")&\
                          ((merged_count_eukulele["Division"]=="Stramenopiles")|\
                           (merged_count_eukulele["Division"]=="NoEUKulele"))].head(n=100)

In [140]:
merged_count_eukulele.loc[(merged_count_eukulele["Family|agg"] == "NoDeepClustWDiatom")&\
                          (merged_count_eukulele["Division"]=="Stramenopiles")]

,Family|agg,Family,Division,Division|agg,transcript_name
58581,NoDeepClustWDiatom,Pelagomonadaceae,Stramenopiles,NoDeepClustWDiatom,56149
58497,NoDeepClustWDiatom,Dictyochales,Stramenopiles,NoDeepClustWDiatom,25962
58518,NoDeepClustWDiatom,Florenciellaceae,Stramenopiles,NoDeepClustWDiatom,15956
58630,NoDeepClustWDiatom,Skeletonemataceae,Stramenopiles,NoDeepClustWDiatom,8997
58544,NoDeepClustWDiatom,Leptocylindraceae,Stramenopiles,NoDeepClustWDiatom,8469
58636,NoDeepClustWDiatom,Stephanodiscaceae,Stramenopiles,NoDeepClustWDiatom,4717
58638,NoDeepClustWDiatom,Stramenopiles_XXXX,Stramenopiles,NoDeepClustWDiatom,2576
58649,NoDeepClustWDiatom,Thalassiosiraceae,Stramenopiles,NoDeepClustWDiatom,2576
58637,NoDeepClustWDiatom,Stephanopyxidaceae,Stramenopiles,NoDeepClustWDiatom,2093
58455,NoDeepClustWDiatom,Chaetocerotales,Stramenopiles,NoDeepClustWDiatom,2011


In [ ]:
### salmon quantification
salmon_all=pd.DataFrame()
salmon_dir="/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic_paper_narragansett/intermediate-files/04-compare/14-MAD-mapping/salmon/"
for curr in glob.glob(os.path.join(salmon_dir,"*/quant.sf")):
    if "_S" not in curr: 
        continue
    salmon_curr = pd.read_csv(curr,sep="\t")
    salmon_curr["Sample"] = curr.split("_quant")[0].split("/")[-1]
    salmon_all=pd.concat([sample_all,salmon_curr])

In [125]:
curr_concat_read_in0_5.loc[curr_concat_read_in0_5.Source_ID=="megahit_NarBay_A_k51_1000010_flag=1_multi=6N2333_len=1401_NarBay_A_merged.fastaNp1"]

,Source_ID,HomClust,cluster_blob,Domain,Supergroup,Division,Class,Order,Family,Genus,Species,Source_ID.1,sample_code


In [118]:
merged_w_eukulele0_5.loc[(merged_w_eukulele0_5.Family!=merged_w_eukulele0_5.Family)&(merged_w_eukulele0_5["Family|lenset"]>1),
                     ["Family|lenset","Division","Domain","Family","Family|set",
                      "Division|set","Class|set","transcript_name"]].head()

,Family|lenset,Division,Domain,Family,Family|set,Division|set,Class|set,transcript_name
30,14.0,Alveolata,Eukaryota,None,"{nan, Peridiniales_X, Goniodomataceae, Amphidomataceae, Ceratiaceae, Kryptoperidiniaceae, Gonyau...","{nan, Dinoflagellata}","{nan, Dinophyceae}",megahit_NarBay_A_k51_1000406_flag=1_multi=4.2866_len=543_NarBay_A_merged.fasta.p2
92,2.0,Stramenopiles,Eukaryota,None,"{nan, Pelagomonadaceae}","{nan, Ochrophyta}","{nan, Pelagophyceae}",megahit_NarBay_A_k51_100120_flag=1_multi=4.0000_len=1175_NarBay_A_merged.fasta.p2
174,31.0,Alveolata,Eukaryota,None,"{nan, Prymnesiaceae, Cyanoptycaceae, Kryptoperidiniaceae, Dictyochophyceae_XX, Gonyaulacaceae, V...","{nan, Ochrophyta, Foraminifera, Haptophyta, Apicomplexa, Sagenista, Glaucophyta, Dinoflagellata,...","{nan, Dinophyceae, Chrysophyceae, Pavlovophyceae, Dictyochophyceae, Prymnesiophyceae, Xanthophyc...",megahit_NarBay_A_k51_1002348_flag=1_multi=1.9716_len=473_NarBay_A_merged.fasta.p2
233,11.0,Alveolata,Eukaryota,None,"{nan, Kryptoperidiniaceae, Amphidomataceae, Gonyaulacaceae, Crypthecodiniaceae, Prorocentraceae,...","{nan, Dinoflagellata}","{nan, Dinophyceae}",megahit_NarBay_A_k51_1003184_flag=0_multi=2.7124_len=357_NarBay_A_merged.fasta.p2
420,19.0,Alveolata,Eukaryota,None,"{nan, Kryptoperidiniaceae, Noelaerhabdaceae, Gonyaulacaceae, Kareniaceae, Peridiniaceae, Goniodo...","{nan, Ochrophyta, Foraminifera, Haptophyta, Dinoflagellata}","{nan, Dinophyceae, Oxyrrhea, Prymnesiophyceae, Tubothalamea, Bacillariophyta}",megahit_NarBay_A_k51_100601_flag=1_multi=2.3390_len=405_NarBay_A_merged.fasta.p1


In [20]:
tester_curr0_5.loc[[np.nan in curr for curr in tester_curr0_5["Division|set"]]].groupby("Division|lenset")["sample_code|sum"].sum().reset_index()

,Division|lenset,sample_code|sum
0,1,2237773
1,2,1166048
2,3,124632
3,4,64364
4,5,19844
5,6,14604
6,7,7915
7,8,9276
8,9,7342
9,10,5808


In [17]:
tax_table=pd.read_csv("/vortexfs1/omics/alexander/data/databases/marmmetsp-5Dec2022/tax-table.txt",sep="\t")
diatom_mmetsps=list(tax_table.loc[tax_table.Family.isin(["Skeletonemataceae","Rhizosoleniaceae",
                                     "Hemiaulaceae","Thalassiosiraceae"]),"Source_ID"])

In [18]:
%%time
build_upset_df0_5=build_upset_df(tester_curr0_5,0.5)

KeyboardInterrupt: 

In [43]:
%%time
curr_concat_read_in0_8,tester_curr0_8,merged_w_eukulele0_8=polish_input_frame(input_dir="/vortexfs1/omics/alexander/akrinos/2022-euk-diversity/code/tax-aliquots/",
                                                                              cluster_threshold=0.8,
                                                                              kmer_no=3,sample_id="NarBay",sample_code="diatomsall")

CPU times: user 5min 33s, sys: 27.1 s, total: 6min
Wall time: 6min


In [91]:
tester_curr0_8.head()

,cluster_blob,HomClust,Division|set,Division|lenset,Division|len,Division|sum,Domain|set,Domain|lenset,Domain|len,Domain|sum,...,Family|len,Family|sum,Class|set,Class|lenset,Class|len,Class|sum,sample_code|set,sample_code|lenset,sample_code|len,sample_code|sum
0,1,HomClust_0,{nan},1,2,0,{nan},1,2,0,...,2,0,{nan},1,2,0,{1},1,2,2
1,1,HomClust_1,{Ochrophyta},1,1,Ochrophyta,{Eukaryota},1,1,Eukaryota,...,1,Odontellaceae,{Bacillariophyta},1,1,Bacillariophyta,{0},1,1,0
2,1,HomClust_10,"{nan, Ochrophyta}",2,15,OchrophytaOchrophyta,"{nan, Eukaryota}",2,15,EukaryotaEukaryota,...,15,ThalassionemataceaeThalassionemataceae,"{nan, Bacillariophyta}",2,15,BacillariophytaBacillariophyta,"{0, 1}",2,15,13
3,1,HomClust_100,{Haptophyta},1,24,HaptophytaHaptophytaHaptophytaHaptophytaHaptop...,{Eukaryota},1,24,EukaryotaEukaryotaEukaryotaEukaryotaEukaryotaE...,...,24,IsochrysidaceaeNoelaerhabdaceaeNoelaerhabdacea...,{Prymnesiophyceae},1,24,PrymnesiophyceaePrymnesiophyceaePrymnesiophyce...,{0},1,24,0
4,1,HomClust_1000,"{nan, Ochrophyta}",2,4,OchrophytaOchrophyta,"{Eukaryota, nan}",2,4,EukaryotaEukaryota,...,4,SkeletonemataceaeSkeletonemataceae,"{nan, Bacillariophyta}",2,4,BacillariophytaBacillariophyta,"{0, 1}",2,4,2


In [93]:
merged_w_eukulele0_8_rev=merged_eukulele.merge(curr_concat_read_in0_8.\
                                  rename({"Source_ID":"Revised_name"},
                                         axis="columns")[["Revised_name","HomClust",
                                                          "cluster_blob"]].\
                                            merge(tester_curr0_8),
                left_on="Revised_name",right_on="Revised_name",how="inner")

In [97]:
merged_w_eukulele0_8_rev[["transcript_name","Revised_name","Family|lenset","Family|set",
                      "Family","Genus","full_classification","transcript_name","sample_code|sum"]].\
    groupby("Family|lenset").transcript_name.count().reset_index()

,Family|lenset,transcript_name,transcript_name
0,1,941724,941724
1,2,539787,539787
2,3,230610,230610
3,4,161672,161672
4,5,98550,98550
5,6,83219,83219
6,7,72333,72333
7,8,67076,67076
8,9,63771,63771
9,10,63785,63785


In [87]:
merged_w_eukulele0_8[["Revised_name","Family|lenset","Family|set",
                      "Family","Genus","full_classification","transcript_name","sample_code|sum"]].\
    loc

,Revised_name,Family|lenset,Family|set,Family,Genus,full_classification,transcript_name,sample_code|sum
14620628,MMETSP0223,18.0,"{nan, Goniodomataceae, Kryptoperidiniaceae, Ce...",NaN,NaN,NaN,NaN,369.0
14620629,MMETSP0223,7.0,"{nan, Amphidomataceae, Prorocentraceae, Soriti...",NaN,NaN,NaN,NaN,447.0
14620630,MMETSP0223,1.0,{Gymnodiniaceae},NaN,NaN,NaN,NaN,0.0
14620631,MMETSP0223,17.0,"{nan, Chrysochromulinaceae, Peridiniales_X, Go...",NaN,NaN,NaN,NaN,1930.0
14620632,MMETSP0223,6.0,"{Euzebyaceae, Pyrocystaceae, Prorocentraceae, ...",NaN,NaN,NaN,NaN,0.0


In [78]:
len(merged_w_eukulele0_8.index)

14620633

In [79]:
len(merged_w_eukulele0_8.drop_duplicates("transcript_name").index)

7624082

In [77]:
merged_w_eukulele0_8.head()

,Unnamed: 0,transcript_name,classification_level,full_classification,classification,max_pid,ambiguous,counts,Domain,Division,...,Family|len,Family|sum,Class|set,Class|lenset,Class|len,Class|sum,sample_code|set,sample_code|lenset,sample_code|len,sample_code|sum
0,0.0,megahit_NarBay_A_k51_1000002_flag=1_multi=3.00...,class,Eukaryota; Amoebozoa; Conosa; Variosea,Variosea,124.0,0.0,NaN,Eukaryota,Amoebozoa,...,9.0,0,{nan},1.0,9.0,0,{1},1.0,9.0,9.0
1,0.0,megahit_NarBay_A_k51_1000010_flag=1_multi=6.23...,order,Eukaryota; Stramenopiles; Ochrophyta; Bacillar...,Leptocylindrales,322.0,0.0,NaN,Eukaryota,Stramenopiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,megahit_NarBay_A_k51_1000013_flag=1_multi=3.85...,class,Eukaryota; Hacrobia; Cryptophyta; Cryptophyceae,Cryptophyceae,109.0,0.0,NaN,Eukaryota,Hacrobia,...,7.0,0,{nan},1.0,7.0,0,{1},1.0,7.0,7.0
3,0.0,megahit_NarBay_A_k51_1000032_flag=0_multi=3.00...,genus,Eukaryota; Hacrobia; Haptophyta; Prymnesiophyc...,Chrysochromulina,493.0,0.0,NaN,Eukaryota,Hacrobia,...,291.0,PrymnesiaceaeChloropicaceaeErythropeltidales_X...,"{nan, Dinophyceae, Prostomatea_1, Colpodea, Cr...",14.0,291.0,PrymnesiophyceaeChloropicophyceaeCompsopogonop...,"{0, 1}",2.0,291.0,225.0
4,0.0,megahit_NarBay_A_k51_1000054_flag=1_multi=2.00...,class,Eukaryota; Stramenopiles; Ochrophyta; Chrysoph...,Chrysophyceae,94.4,0.0,NaN,Eukaryota,Stramenopiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
tester_curr0_8.loc[[np.nan in curr for curr in tester_curr0_8["Family|set"]]].groupby("Family|lenset")["sample_code|sum"].sum().reset_index()

,Family|lenset,sample_code|sum
0,1,941703
1,2,539765
2,3,230598
3,4,161665
4,5,98545
5,6,83215
6,7,72328
7,8,67074
8,9,63762
9,10,63781


In [45]:
tester_curr0_8.loc[[np.nan in curr for curr in tester_curr0_8["Division|set"]]].groupby("Division|lenset")["sample_code|sum"].sum().reset_index()

,Division|lenset,sample_code|sum
0,1,941694
1,2,1193103
2,3,380490
3,4,294912
4,5,139048
5,6,101505
6,7,88360
7,8,61041
8,9,61105
9,10,55285


In [103]:
compare_agg0_8=get_compare_agg(merged_w_eukulele0_8,sample_id="NarBay")



<ipython-input-101-568dae809e13>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<ipython-input-101-568dae809e13>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [104]:
compare_agg0_8.head()

,FamilyInFill,Family|agg,transcript_name,in_clust
219532,Unannotated,,756368,no
176112,Skeletonemataceae,Skeletonemataceae,124110,"yes,one"
176148,Skeletonemataceae,Skeletonemataceae_Thalassiosiraceae,45046,"yes,two"
152668,Prorocentraceae,Prorocentraceae,32307,"yes,one"
176128,Skeletonemataceae,Skeletonemataceae_Stephanodiscaceae_Thalassios...,29173,"yes,two"


In [105]:
compare_agg0_8.loc[compare_agg0_8.FamilyInFill=="Unannotated"].groupby("in_clust").transcript_name.sum().reset_index()

,in_clust,transcript_name
0,no,756368
1,"yes,one",158123
2,"yes,two",511314


In [50]:
compare_agg0_8.loc[compare_agg0_8.FamilyInFill=="Unannotated"].groupby("in_clust").transcript_name.sum().reset_index()

,in_clust,transcript_name
0,no,756368
1,"yes,one",158123
2,"yes,two",511314


In [60]:
compare_agg0_8.transcript_name.sum()

3692119

### Let's see how many more diatom sequences are annotated when we add the tax-aliquots approach

In [66]:
## Sequences that have EUKulele or tax-aliquots annotations
compare_agg0_8.loc[(compare_agg0_8.FamilyInFill!="Unannotated")|(compare_agg0_8["Family|agg"]!=""),"transcript_name"].sum()

2935751

In [67]:
2935751/3692119

0.7951398641268063

In [64]:
## Sequences that have EUKulele annotations
compare_agg0_8.loc[(compare_agg0_8.FamilyInFill!="Unannotated")].transcript_name.sum()

2266314

In [65]:
2266314/3692119

0.6138247440020216

In [68]:
## Sequences that have tax-aliquots annotations
compare_agg0_8.loc[(compare_agg0_8["Family|agg"]!="")].transcript_name.sum()

2750395

In [108]:
print(tester_curr0_8.loc[[np.nan in curr for curr in tester_curr0_8["Family|set"]]]["sample_code|sum"].sum())
tester_curr0_8.loc[[np.nan in curr for curr in tester_curr0_8["Family|set"]]].groupby("Family|lenset")["sample_code|sum"].sum().reset_index()

3691965


,Family|lenset,sample_code|sum
0,1,941703
1,2,539765
2,3,230598
3,4,161665
4,5,98545
5,6,83215
6,7,72328
7,8,67074
8,9,63762
9,10,63781


In [115]:
pd.set_option('max.colwidth', 100)
merged_w_eukulele0_8_rev.loc[(merged_w_eukulele0_8_rev.Family!=merged_w_eukulele0_8_rev.Family)&(merged_w_eukulele0_8_rev["Family|lenset"]>1),
                             ["Family|lenset","Division","Domain","Family","Family|set",
                              "Division|set","Class|set","transcript_name"]]

,Family|lenset,Division,Domain,Family,Family|set,Division|set,Class|set,transcript_name
19,16,Alveolata,Eukaryota,None,"{nan, Peridiniales_X, Kryptoperidiniaceae, Ceratiaceae, Amphidomataceae, Naviculaceae, Gonyaulac...","{nan, Dinoflagellata, Ochrophyta}","{nan, Dinophyceae, Bacillariophyta}",megahit_NarBay_A_k51_1000406_flag=1_multi=4.2866_len=543_NarBay_A_merged.fasta.p2
22,6,Rhizaria,Eukaryota,None,"{nan, Chrysochromulinaceae, Goniomonadales_X, Thecofilosea_XX, Chlorarachnida_X, Cafeteriaceae}","{nan, Haptophyta, Cercozoa, Cryptophyta, Opalozoa}","{Bicoecea, nan, Cryptophyceae, Chlorarachniophyceae, Prymnesiophyceae, Thecofilosea}",megahit_NarBay_A_k51_1000444_flag=1_multi=3.0000_len=704_NarBay_A_merged.fasta.p1
23,2,Archaeplastida,Eukaryota,None,"{nan, Pyramimonadales_XX}","{nan, Chlorophyta}","{nan, Pyramimonadales}",megahit_NarBay_A_k51_1000449_flag=1_multi=3.0000_len=383_NarBay_A_merged.fasta.p2
29,19,Stramenopiles,Eukaryota,None,"{nan, Kryptoperidiniaceae, Fragilariaceae, Thalassiosiraceae, Staurosiraceae, Thalassionematacea...","{nan, Dinoflagellata, Ochrophyta, Ciliophora}","{nan, Dinophyceae, Prostomatea_1, Chrysophyceae, Bacillariophyta}",megahit_NarBay_A_k51_10005_flag=1_multi=93.7597_len=4349_NarBay_A_merged.fasta.p1
30,3,Stramenopiles,Eukaryota,None,"{nan, Leptocylindraceae, Bacillariaceae}","{nan, Ochrophyta}","{nan, Bacillariophyta}",megahit_NarBay_A_k51_1000611_flag=1_multi=56.5167_len=800_NarBay_A_merged.fasta.p1
...,...,...,...,...,...,...,...,...
3692096,2,NaN,NaN,NaN,"{nan, Stephanodiscaceae}","{nan, Ochrophyta}","{nan, Bacillariophyta}",trinity_NarBay_S5_TRINITY_DN9855_c0_g1_i13_len=715_path=[0:0-81_3:82-173_4:174-183_5:184-196_6:1...
3692097,2,NaN,NaN,NaN,"{nan, Stephanodiscaceae}","{nan, Ochrophyta}","{nan, Bacillariophyta}",trinity_NarBay_S5_TRINITY_DN9855_c0_g1_i1_len=710_path=[2:0-91_3:92-183_4:184-193_5:194-206_7:20...
3692099,3,NaN,NaN,NaN,"{nan, Thalassiosiraceae, Skeletonemataceae}","{nan, Ochrophyta}","{nan, Bacillariophyta}",trinity_NarBay_S5_TRINITY_DN9893_c0_g3_i1_len=352_path=[0:0-351]_NarBay_S5_merged.fasta.p1
3692110,2,NaN,NaN,NaN,"{Striatellaceae, nan}","{nan, Ochrophyta}","{nan, Bacillariophyta}",trinity_NarBay_S5_TRINITY_DN992_c0_g1_i7_len=367_path=[2:0-28_3:29-86_4:87-146_5:147-206_6:207-2...


In [109]:
3691965-941703

2750262

In [71]:
compare_agg0_8.head()

,FamilyInFill,Family|agg,transcript_name,in_clust
219532,Unannotated,,756368,no
176112,Skeletonemataceae,Skeletonemataceae,124110,"yes,one"
176148,Skeletonemataceae,Skeletonemataceae_Thalassiosiraceae,45046,"yes,two"
152668,Prorocentraceae,Prorocentraceae,32307,"yes,one"
176128,Skeletonemataceae,Skeletonemataceae_Stephanodiscaceae_Thalassios...,29173,"yes,two"


In [70]:
compare_agg0_8.loc[(compare_agg0_8["Family|agg"]!="")].groupby("in_clust").transcript_name.sum()

in_clust
yes,one     497754
yes,two    2252641
Name: transcript_name, dtype: int64

In [51]:
756368+158123+511314

1425805

In [52]:
%%time
curr_concat_read_in0_2,tester_curr0_2,merged_w_eukulele0_2=polish_input_frame(input_dir="/vortexfs1/omics/alexander/akrinos/2022-euk-diversity/code/tax-aliquots/",
                                                                              cluster_threshold=0.2,
                                                                              kmer_no=3,sample_id="NarBay",sample_code="diatomsall")

CPU times: user 17min, sys: 1min, total: 18min
Wall time: 18min 3s


In [155]:
def aggset(input_set):
    if input_set!=input_set:
        return "NoDeepClustWDiatom"
    return("_".join(sorted(list(set([str(curr) for curr in input_set if curr==curr])))))

merged_w_eukulele0_2["Family|agg"] = [aggset(curr) for curr in merged_w_eukulele0_2["Family|set"]]
merged_w_eukulele0_2["Division|agg"] = [aggset(curr) for curr in merged_w_eukulele0_2["Division|set"]]

merged_w_eukulele0_2[["Family|agg","Family|lenset","Division","Domain","Family","Family|set",
                      "Division|set","Class|set","transcript_name"]].head()

,Family|agg,Family|lenset,Division,Domain,Family,Family|set,Division|set,Class|set,transcript_name
0,,1.0,Amoebozoa,Eukaryota,None,{nan},{nan},{nan},megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta.p2
1,NoDeepClustWDiatom,NaN,Stramenopiles,Eukaryota,None,NaN,NaN,NaN,megahit_NarBay_A_k51_1000010_flag=1_multi=6.2333_len=1401_NarBay_A_merged.fasta.p1
2,,1.0,Hacrobia,Eukaryota,None,{nan},{nan},{nan},megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta.p1
3,,1.0,Hacrobia,Eukaryota,Chrysochromulinaceae,{nan},{nan},{nan},megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta.p1
4,NoDeepClustWDiatom,NaN,Stramenopiles,Eukaryota,None,NaN,NaN,NaN,megahit_NarBay_A_k51_1000054_flag=1_multi=2.0000_len=353_NarBay_A_merged.fasta.p1


In [156]:
merged_count_eukulele0_2 = merged_w_eukulele0_2.groupby(["Division|agg","Family|agg","Division","Family"])\
    ["transcript_name"].\
    count().reset_index().sort_values("transcript_name",ascending=False)
merged_count_eukulele0_2.loc[merged_count_eukulele0_2["Family|agg"] != merged_count_eukulele0_2["Family"]]

merged_count_eukulele0_2.loc[merged_count_eukulele0_2["Family|agg"] != merged_count_eukulele0_2["Family"]].head(n=100)

,Division|agg,Family|agg,Division,Family,transcript_name
8934,NoDeepClustWDiatom,NoDeepClustWDiatom,Alveolata,Prorocentraceae,487078
254,,,Stramenopiles,Skeletonemataceae,142420
57,,,Alveolata,Suessiaceae,124836
237,,,Stramenopiles,Hemiaulaceae,111380
8939,NoDeepClustWDiatom,NoDeepClustWDiatom,Alveolata,Suessiaceae,92776
222,,,Stramenopiles,Chaetocerotales,89002
263,,,Stramenopiles,Thalassiosiraceae,79865
121,,,Hacrobia,Chrysochromulinaceae,70855
52,,,Alveolata,Prorocentraceae,63305
239,,,Stramenopiles,Leptocylindraceae,60221


In [158]:
merged_count_eukulele0_2.loc[["Ochrophyta" in curr for curr in merged_count_eukulele0_2["Division|agg"]] |\
                             (merged_count_eukulele0_2["Division"]=="Stramenopiles")].head(n=100)

,Division|agg,Family|agg,Division,Family,transcript_name
11151,Ochrophyta,Skeletonemataceae,Stramenopiles,Skeletonemataceae,269161
254,,,Stramenopiles,Skeletonemataceae,142420
237,,,Stramenopiles,Hemiaulaceae,111380
222,,,Stramenopiles,Chaetocerotales,89002
263,,,Stramenopiles,Thalassiosiraceae,79865
239,,,Stramenopiles,Leptocylindraceae,60221
9094,NoDeepClustWDiatom,NoDeepClustWDiatom,Stramenopiles,Pelagomonadaceae,56147
257,,,Stramenopiles,Stephanodiscaceae,41794
11022,Ochrophyta,Pelagomonadaceae,Stramenopiles,Pelagomonadaceae,41470
248,,,Stramenopiles,Pelagomonadaceae,31460


In [161]:
import glob
### salmon quantification
salmon_all=pd.DataFrame()
salmon_dir="/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic_paper_narragansett/intermediate-files/04-compare/14-MAD-mapping/salmon/"
for curr in glob.glob(os.path.join(salmon_dir,"*/quant.sf")):
    if "_S" not in curr: 
        continue
    salmon_curr = pd.read_csv(curr,sep="\t")
    salmon_curr["Sample"] = curr.split("_quant")[0].split("/")[-1]
    salmon_all=pd.concat([salmon_all,salmon_curr])

In [167]:
merged_w_eukulele0_2["Name"] = [str(curr).split(".p")[0] for curr in merged_w_eukulele0_2["transcript_name"]]

merged_w_eukulele0_2_cts=merged_w_eukulele0_2.loc[merged_w_eukulele0_2["Family|agg"] == \
                                                  merged_w_eukulele0_2["Family"]].\
    merge(salmon_all[["Name","TPM","NumReads","Sample"]],how="left",left_on="Name",right_on="Name")

,Unnamed: 0,transcript_name,classification_level,full_classification,classification,max_pid,ambiguous,counts,Domain,Division,...,sample_code|len,sample_code|sum,Family|agg,Division|agg,Name,Length,EffectiveLength,TPM,NumReads,Sample
0,0.0,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta.p3,species,Eukaryota; Stramenopiles; Ochrophyta; Pelagophyceae; Pelagomonadales; Pelagomonadaceae; Aureococ...,Aureococcus anophagefferens,360.0,0.0,NaN,Eukaryota,Stramenopiles,...,5.0,4.0,Pelagomonadaceae,Ochrophyta,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta,570.0,389.419,0.000000,0.000,NarBay_S4
1,0.0,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta.p3,species,Eukaryota; Stramenopiles; Ochrophyta; Pelagophyceae; Pelagomonadales; Pelagomonadaceae; Aureococ...,Aureococcus anophagefferens,360.0,0.0,NaN,Eukaryota,Stramenopiles,...,5.0,4.0,Pelagomonadaceae,Ochrophyta,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta,570.0,384.661,0.000000,0.000,NarBay_S2
2,0.0,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta.p3,species,Eukaryota; Stramenopiles; Ochrophyta; Pelagophyceae; Pelagomonadales; Pelagomonadaceae; Aureococ...,Aureococcus anophagefferens,360.0,0.0,NaN,Eukaryota,Stramenopiles,...,5.0,4.0,Pelagomonadaceae,Ochrophyta,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta,570.0,398.086,0.000000,0.000,NarBay_S1
3,0.0,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta.p3,species,Eukaryota; Stramenopiles; Ochrophyta; Pelagophyceae; Pelagomonadales; Pelagomonadaceae; Aureococ...,Aureococcus anophagefferens,360.0,0.0,NaN,Eukaryota,Stramenopiles,...,5.0,4.0,Pelagomonadaceae,Ochrophyta,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta,570.0,403.165,0.043477,4.859,NarBay_S3
4,0.0,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta.p3,species,Eukaryota; Stramenopiles; Ochrophyta; Pelagophyceae; Pelagomonadales; Pelagomonadaceae; Aureococ...,Aureococcus anophagefferens,360.0,0.0,NaN,Eukaryota,Stramenopiles,...,5.0,4.0,Pelagomonadaceae,Ochrophyta,megahit_NarBay_A_k51_100012_flag=1_multi=2.7534_len=570_NarBay_A_merged.fasta,570.0,389.105,0.000000,0.000,NarBay_S5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048478,0.0,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,species,Eukaryota; Stramenopiles; Ochrophyta; Bacillariophyta; Thalassiosirales; Skeletonemataceae; Skel...,Skeletonema marinoi,747.0,0.0,NaN,Eukaryota,Stramenopiles,...,28.0,27.0,Skeletonemataceae,Ochrophyta,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,1380.0,1199.252,0.156450,18.502,NarBay_S4
3048479,0.0,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,species,Eukaryota; Stramenopiles; Ochrophyta; Bacillariophyta; Thalassiosirales; Skeletonemataceae; Skel...,Skeletonema marinoi,747.0,0.0,NaN,Eukaryota,Stramenopiles,...,28.0,27.0,Skeletonemataceae,Ochrophyta,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,1380.0,1194.371,0.297736,63.708,NarBay_S2
3048480,0.0,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,species,Eukaryota; Stramenopiles; Ochrophyta; Bacillariophyta; Thalassiosirales; Skeletonemataceae; Skel...,Skeletonema marinoi,747.0,0.0,NaN,Eukaryota,Stramenopiles,...,28.0,27.0,Skeletonemataceae,Ochrophyta,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,1380.0,1207.972,0.263285,98.151,NarBay_S1
3048481,0.0,trinity_NarBay_S5_TRINITY_DN9996_c0_g1_i4_len=1380_path=[1:0-193_5:194-272_7:273-455_8:456-608_9...,species,Eukaryota; Stramenopiles; Ochrophyta; Bacillariophyta; Thalassiosirales; Skeleto

In [168]:
merged_w_eukulele0_2_cts=merged_w_eukulele0_2.loc[merged_w_eukulele0_2["Family|agg"] == \
                                                  merged_w_eukulele0_2["Family"]].\
    merge(salmon_all[["Name","TPM","NumReads","Sample"]],how="left",left_on="Name",right_on="Name")

In [173]:
merged_w_eukulele0_2_cts.groupby(["Family|agg","Sample"]).TPM.sum().reset_index().sort_values("TPM",
                                                                                             ascending=False)

,Family|agg,Sample,TPM
531,Skeletonemataceae,NarBay_S2,95500.343198
530,Skeletonemataceae,NarBay_S1,37264.691468
532,Skeletonemataceae,NarBay_S3,8304.915985
487,Prorocentraceae,NarBay_S3,7654.993741
299,Hemiaulaceae,NarBay_S5,5499.186725
550,Stephanodiscaceae,NarBay_S1,5286.080988
533,Skeletonemataceae,NarBay_S4,5060.942008
298,Hemiaulaceae,NarBay_S4,4487.107643
417,Pelagomonadaceae,NarBay_S3,3606.202704
297,Hemiaulaceae,NarBay_S3,3174.415985


In [170]:
merged_w_eukulele0_2_cts.groupby("classification_level").transcript_name.count().reset_index()

,classification_level,transcript_name
0,family,45480
1,genus,714166
2,species,2288837


In [177]:
merged_w_eukulele0_5.head()

,Unnamed: 0,transcript_name,classification_level,full_classification,classification,max_pid,ambiguous,counts,Domain,Division,...,Class|lenset,Class|len,Class|sum,sample_code|set,sample_code|lenset,sample_code|len,sample_code|sum,Family|agg,Division|agg,Name
0,0.0,megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta.p2,class,Eukaryota; Amoebozoa; Conosa; Variosea,Variosea,124.0,0.0,NaN,Eukaryota,Amoebozoa,...,1.0,9.0,0,{1},1.0,9.0,9.0,,,megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta
1,0.0,megahit_NarBay_A_k51_1000010_flag=1_multi=6.2333_len=1401_NarBay_A_merged.fasta.p1,order,Eukaryota; Stramenopiles; Ochrophyta; Bacillariophyta; Leptocylindrales,Leptocylindrales,322.0,0.0,NaN,Eukaryota,Stramenopiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoDeepClustWDiatom,NoDeepClustWDiatom,megahit_NarBay_A_k51_1000010_flag=1_multi=6.2333_len=1401_NarBay_A_merged.fasta
2,0.0,megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta.p1,class,Eukaryota; Hacrobia; Cryptophyta; Cryptophyceae,Cryptophyceae,109.0,0.0,NaN,Eukaryota,Hacrobia,...,1.0,7.0,0,{1},1.0,7.0,7.0,,,megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta
3,0.0,megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta.p1,genus,Eukaryota; Hacrobia; Haptophyta; Prymnesiophyceae; Prymnesiales; Chrysochromulinaceae; Chrysochr...,Chrysochromulina,493.0,0.0,NaN,Eukaryota,Hacrobia,...,1.0,8.0,0,{1},1.0,8.0,8.0,,,megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta
4,0.0,megahit_NarBay_A_k51_1000054_flag=1_multi=2.0000_len=353_NarBay_A_merged.fasta.p1,class,Eukaryota; Stramenopiles; Ochrophyta; Chrysophyceae,Chrysophyceae,94.4,0.0,NaN,Eukaryota,Stramenopiles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoDeepClustWDiatom,NoDeepClustWDiatom,megahit_NarBay_A_k51_1000054_flag=1_multi=2.0000_len=353_NarBay_A_merged.fasta


In [174]:
merged_w_eukulele0_5["Name"] = [str(curr).split(".p")[0] for curr in merged_w_eukulele0_5["transcript_name"]]

merged_w_eukulele0_5_cts=merged_w_eukulele0_5.loc[merged_w_eukulele0_5["Family|lenset"] == \
                                                  merged_w_eukulele0_5["Family"]].\
    merge(salmon_all[["Name","TPM","NumReads","Sample"]],how="left",left_on="Name",right_on="Name")

In [192]:
merged_w_eukulele0_5_coherent=merged_w_eukulele0_5.loc[merged_w_eukulele0_5["Family|lenset"] == 1].\
    merge(salmon_all[["Name","TPM","NumReads","Sample"]],how="left",left_on="Name",right_on="Name")

MemoryError: Unable to allocate 1.37 GiB for an array with shape (26, 7086651) and data type object

In [187]:
import gc
gc.collect()

40

In [186]:
%xdel all_sequences_nb

In [188]:
%xdel EUKulele_MAD_cds_marmmetsp

In [190]:
%xdel merged_w_eukulele0_8_rev

In [182]:
%whos

Variable                     Type             Data/Info
-------------------------------------------------------
AgglomerativeClustering      type             <class 'sklearn.cluster._<...>AgglomerativeClustering'>
Automaton                    type             <class 'ahocorasick.Automaton'>
Client                       type             <class 'distributed.client.Client'>
EUKulele_MAD_cds_marmmetsp   DataFrame                 Unnamed: 0      <...>5817409 rows x 8 columns]
SeqIO                        module           <module 'Bio.SeqIO' from <...>s/Bio/SeqIO/__init__.py'>
SequenceMatcher              type             <class 'difflib.SequenceMatcher'>
aggset                       function         <function aggset at 0x2ab3d7c7c0d0>
ahocorasick                  module           <module 'ahocorasick' fro<...>-38-x86_64-linux-gnu.so'>
all_sequences_nb             DataFrame                                 <...>7624081 rows x 1 columns]
behaved_fasta                str              nb_fasta_f

In [183]:
merged_w_eukulele0_5.loc[merged_w_eukulele0_5["Family|lenset"] == 1]

,Unnamed: 0,transcript_name,classification_level,full_classification,classification,max_pid,ambiguous,counts,Domain,Division,...,Class|lenset,Class|len,Class|sum,sample_code|set,sample_code|lenset,sample_code|len,sample_code|sum,Family|agg,Division|agg,Name
0,0.0,megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta.p2,class,Eukaryota; Amoebozoa; Conosa; Variosea,Variosea,124.0,0.0,NaN,Eukaryota,Amoebozoa,...,1.0,9.0,0,{1},1.0,9.0,9.0,,,megahit_NarBay_A_k51_1000002_flag=1_multi=3.0000_len=453_NarBay_A_merged.fasta
2,0.0,megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta.p1,class,Eukaryota; Hacrobia; Cryptophyta; Cryptophyceae,Cryptophyceae,109.0,0.0,NaN,Eukaryota,Hacrobia,...,1.0,7.0,0,{1},1.0,7.0,7.0,,,megahit_NarBay_A_k51_1000013_flag=1_multi=3.8537_len=461_NarBay_A_merged.fasta
3,0.0,megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta.p1,genus,Eukaryota; Hacrobia; Haptophyta; Prymnesiophyceae; Prymnesiales; Chrysochromulinaceae; Chrysochr...,Chrysochromulina,493.0,0.0,NaN,Eukaryota,Hacrobia,...,1.0,8.0,0,{1},1.0,8.0,8.0,,,megahit_NarBay_A_k51_1000032_flag=0_multi=3.0062_len=853_NarBay_A_merged.fasta
7,0.0,megahit_NarBay_A_k51_1000085_flag=1_multi=3.0000_len=595_NarBay_A_merged.fasta.p1,order,Eukaryota; Alveolata; Dinoflagellata; Dinophyceae; Suessiales,Suessiales,112.0,0.0,NaN,Eukaryota,Alveolata,...,1.0,1.0,0,{1},1.0,1.0,1.0,,,megahit_NarBay_A_k51_1000085_flag=1_multi=3.0000_len=595_NarBay_A_merged.fasta
8,0.0,megahit_NarBay_A_k51_100009_flag=1_multi=1.8148_len=591_NarBay_A_merged.fasta.p1,class,Eukaryota; Rhizaria; Cercozoa; Chlorarachniophyceae,Chlorarachniophyceae,114.0,0.0,NaN,Eukaryota,Rhizaria,...,1.0,2.0,0,{1},1.0,2.0,2.0,,,megahit_NarBay_A_k51_100009_flag=1_multi=1.8148_len=591_NarBay_A_merged.fasta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14619267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoEUKulele,...,1.0,1.0,Dinophyceae,{0},1.0,1.0,0.0,Prorocentraceae,Dinoflagellata,nan
14619268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoEUKulele,...,1.0,1.0,Dinophyceae,{0},1.0,1.0,0.0,Prorocentraceae,Dinoflagellata,nan
14619269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoEUKulele,...,1.0,1.0,Dinophyceae,{0},1.0,1.0,0.0,Prorocentraceae,Dinoflagellata,nan
14619271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoEUKulele,...,1.0,1.0,Dinophyceae,{0},1.0,1.0,0.0,Prorocentraceae,Dinoflagellata,nan


In [175]:
merged_w_eukulele0_5_cts.groupby("classification_level").transcript_name.count().reset_index()

,classification_level,transcript_name
0,family,344840
1,genus,1191571
2,species,1924702


In [176]:
merged_w_eukulele0_5_cts.groupby(["Family|agg","Sample"]).TPM.sum().reset_index().sort_values("TPM",
                                                                                             ascending=False)

,Family|agg,Sample,TPM
661,Skeletonemataceae,NarBay_S2,77319.289878
660,Skeletonemataceae,NarBay_S1,19156.640141
354,Hemiaulaceae,NarBay_S5,15764.913253
353,Hemiaulaceae,NarBay_S4,15317.492062
352,Hemiaulaceae,NarBay_S3,6234.245006
582,Prorocentraceae,NarBay_S3,6112.976909
662,Skeletonemataceae,NarBay_S3,5668.175388
680,Stephanodiscaceae,NarBay_S1,3654.206986
663,Skeletonemataceae,NarBay_S4,3602.606599
392,Leptocylindraceae,NarBay_S3,2868.444103


In [53]:
tester_curr0_2.loc[[np.nan in curr for curr in tester_curr0_2["Division|set"]]].groupby("Division|lenset")["sample_code|sum"].sum().reset_index()

,Division|lenset,sample_code|sum
0,1,2950766
1,2,704079
2,3,19052
3,4,9977
4,5,2275
5,6,1886
6,7,1367
7,8,530
8,9,507
9,10,353
